In [ ]:
import utilities
import os
import socket

In [ ]:
Traffic = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Traffic/Traffic_echo_0307"
categories = utilities.get_directories_in_a_directory(Traffic)
base_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/post_processed/encrypted_echo/"

### DNS Resolutions

In [ ]:
# passed = ["Fashion-Style", "ConnectedCar","SmartHome", "Wine-Beverages", "Pets-Animals", "Navigation-TripPlanners",
#          "Religion-Spirituality", "Dating", "Health-Fitness"]
for category in categories:
    #print(category)
    dns_resolutions = {}
    category_name = category.split('/')[-1]
    if category_name in passed:
        continue
    print(category)
    network_json_dir = os.path.join(Traffic,category_name, category_name+"-ENC-out.json")
    network_json = utilities.read_json(network_json_dir)
    
    for i in range(len(network_json)):
        if network_json[i]["_source"]["layers"]["frame"]["frame.protocols"] == 'eth:ethertype:ip:udp:dns':
            answers_key = []
            if "Answers" in network_json[i]["_source"]["layers"]["dns"].keys():
#                 answers_key  = network_json[i]["_source"]["layers"]["dns"]["Answers"].keys()
#                 for key in answers_key:
#                     if "CNAME" in key:
#                         #print("CNAME")
#                         continue
                    dns_a = network_json[i]["_source"]["layers"]["dns"]["Answers"][key]["dns.a"]
                    dns_name = network_json[i]["_source"]["layers"]["dns"]["Answers"][key]["dns.resp.name"]
                    dns_resolutions[dns_a] = dns_name
    utilities.write_json(os.path.join(category, "dns_resolutions.json"), dns_resolutions)
                

### Aggregate DNS Resolutions

In [ ]:
general_dns_resolutions = {}
for category in categories:
    category_name = category.split('/')[-1]
    dns_resolutions = utilities.read_json(os.path.join(Traffic, category_name, "dns_resolutions.json"))
    for ip, dns_name in dns_resolutions.items():
        if ip not in general_dns_resolutions:
            general_dns_resolutions[ip] = dns_name
    print(category)
utilities.write_json(os.path.join(base_dir, "general_dns_resolutions.json"), general_dns_resolutions)

### Source & Destination IPs

In [ ]:
passed = ["Fashion-Style", "SmartHome", "Wine-Beverages", "Pets-Animals","Navigation-TripPlanners",
         "Religion-Spirituality", "Dating", "Health-Fitness"]
for category in categories:
    
    ip = {"src": [], "dst": []}
    category_name = category.split('/')[-1]
    if category_name in passed:
        continue
    print(category)
    network_json_dir = os.path.join(category, category_name+"-ENC-out.json")
    network_json = utilities.read_json(network_json_dir)
    
    for i in range(len(network_json)):
        if 'ip' in network_json[i]["_source"]["layers"].keys():
            ip["src"].append(network_json[i]["_source"]["layers"]['ip']['ip.src'])
            ip["dst"].append(network_json[i]["_source"]["layers"]['ip']['ip.dst'])
    
    utilities.write_json(os.path.join(Traffic, category_name, "ip.json"), ip)

# Skill Analysis

In [ ]:
#general_dns_resolutions = utilities.read_json(os.path.join(base_dir, "general_dns_resolutions.json"))
general_dns_resolutions = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/post_processed/encrypted_echo/general_dns_resolutions.json")

In [ ]:
print(len(general_dns_resolutions))

In [ ]:
### Filter Lists
advertisers = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists/advertisers.json")
trackers = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists/trackers.json")

In [ ]:
all_non_resolvable_domains = set()

# Fashion-Style

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Fashion-Style", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

In [ ]:
all_non_resolvable_domains

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Connected Car

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "ConnectedCar", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

In [ ]:
all_non_resolvable_domains

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Dating

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Dating", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
        all_non_resolvable_domains.add(ip)
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Health Fitness

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Health-Fitness", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Navigation TripPlanners

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Navigation-TripPlanners", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Pets Animals

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Pets-Animals", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Religion Sprituality

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Religion-Spirituality", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Smart Homes

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "SmartHome", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Wine Beverages

In [ ]:
ips = utilities.read_json(os.path.join(Traffic, "Wine-Beverages", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

### Tracking domains

In [ ]:
for src in src_domains:
    if src in trackers:
        print(src)
for dst in dst_domains:
    if dst in advertisers:
        print(dst)

### non-resolvable domains

In [ ]:
print(non_resolvable_dst.union(non_resolvable_src))

# Non-resolvable IPs

In [ ]:
all_non_resolvable_domains
utilities.write_json(os.path.join(base_dir, "all_non_resolvable_domains.json"), list(all_non_resolvable_domains))

In [ ]:
data_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data"
all_non_resolvable_domains = utilities.read_json(os.path.join(base_dir, "all_non_resolvable_domains.json"))
gethostbysocket = {}
all_non_resolvable_domains_new = set()

### Address other non-resolvable IPs

In [ ]:
import socket
for ip in all_non_resolvable_domains:
    try:
        gethostbysocket[ip] = socket.gethostbyaddr(ip)[0]
    except:
        all_non_resolvable_domains_new.add(ip)
data_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data"        
utilities.write_json(os.path.join(base_dir, "all_non_resolvable_domains_new.json"), list(all_non_resolvable_domains_new))
utilities.write_json(os.path.join(base_dir, "other_dns_resolutions.json"), gethostbysocket)

In [ ]:
filter_list_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists"
tracking_domains = utilities.read_json(os.path.join(filter_list_dir, "tracking_domains.json"))
for ip, host in gethostbysocket.items():
    #print(ip, host)
    if host in tracking_domains:
        print(host)

In [ ]:
non_resolvable_ips = []
for ip in all_non_resolvable_domains:
    if ip not in general_dns_resolutions.keys():
        non_resolvable_ips.append(ip)
utilities.write_json(os.path.join(base_dir, "non_resolvable_ips.json"), list(non_resolvable_ips))


In [ ]:
#non_resolvable IPs
non_obs_ips = utilities.read_json(os.path.join(data_dir, "all_non_resolvable_domains_new.json"))
non_obs_ips